In [1]:
import sqlite3

conn = sqlite3.connect('./quant_db.sqlite', isolation_level= None)
c = conn.cursor()

In [2]:
query = """
CREATE TABLE if NOT EXISTS kor_value
(
    종목코드 varchar(6),
    기준일 date,
    지표 varchar(3),
    값 double,
    primary key (종목코드, 기준일, 지표)
);
"""
c.execute(query)

In [3]:
import pandas as pd
import numpy as np

# 분기 재무제표 불러오기
kor_fs = pd.read_sql("""
select * from kor_fs
where 공시구분 = 'q'
and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=conn)

# 티커 리스트 불러오기
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
and 종목구분 = '보통주';
""", con=conn)

In [4]:
# TTM 구하기
kor_fs = kor_fs.sort_values(['종목코드', '계정', '기준일'])
kor_fs['ttm'] = kor_fs.groupby(['종목코드', '계정'], as_index=False)['값'].rolling(
    window=4, min_periods=4).sum()['값']

# 자본은 평균 구하기
kor_fs['ttm'] = np.where(kor_fs['계정'] == '자본', kor_fs['ttm'] / 4,
                         kor_fs['ttm'])
kor_fs = kor_fs.groupby(['계정', '종목코드']).tail(1)

In [5]:
kor_fs_merge = kor_fs[['계정', '종목코드',
                       'ttm']].merge(ticker_list[['종목코드', '시가총액', '기준일']],
                                     on='종목코드')
kor_fs_merge['시가총액'] = kor_fs_merge['시가총액'] / 100000000

kor_fs_merge['value'] = kor_fs_merge['시가총액'] / kor_fs_merge['ttm']
kor_fs_merge['value'] = kor_fs_merge['value'].round(4)
kor_fs_merge['지표'] = np.where(
    kor_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        kor_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(kor_fs_merge['계정'] == '자본', 'PBR',
                 np.where(kor_fs_merge['계정'] == '당기순이익', 'PER', None))))

kor_fs_merge.rename(columns={'value': '값'}, inplace=True)
kor_fs_merge = kor_fs_merge[['종목코드', '기준일', '지표', '값']]
kor_fs_merge = kor_fs_merge.replace([np.inf, -np.inf, np.nan], None)

kor_fs_merge.head(4)

,종목코드,기준일,지표,값
0,000020,2023-09-19,PER,9.4948
1,000020,2023-09-19,PSR,0.7917
2,000020,2023-09-19,PCR,9.4315
3,000020,2023-09-19,PBR,0.7386


In [6]:
query = """
    INSERT or IGNORE INTO kor_value (종목코드, 기준일, 지표, 값)
    values (?,?,?,?);
"""

args_fs = kor_fs_merge.values.tolist()
c.executemany(query, args_fs)

In [7]:
ticker_list['값'] = ticker_list['주당배당금'] / ticker_list['종가']
ticker_list['값'] = ticker_list['값'].round(4)
ticker_list['지표'] = 'DY'
dy_list = ticker_list[['종목코드', '기준일', '지표', '값']]
dy_list = dy_list.replace([np.inf, -np.inf, np.nan], None)
dy_list = dy_list[dy_list['값'] != 0]

dy_list.head()

,종목코드,기준일,지표,값
0,095570,2023-09-19,DY,0.0605
1,006840,2023-09-19,DY,0.0107
2,027410,2023-09-19,DY,0.0303
3,282330,2023-09-19,DY,0.0275
4,138930,2023-09-19,DY,0.0867


In [8]:
args_dy = dy_list.values.tolist()
c.executemany(query, args_dy)